# Allowing storage of yaml file

Here we will correct the model so that it can be stored in `.yml` format, and do some tests to check all is in place.

Benjamín J. Sánchez, 2020-05-06

## 1. Non-compliant notes

In [1]:
import cobra
model = cobra.io.read_sbml_model("../model/g-thermo.xml")
cobra.io.save_yaml_model(model,"../model/g-thermo.yml")

'alpha-D-Galactosyl-(1⇢3)-1D-myo-inositol;'


UnicodeEncodeError: 'charmap' codec can't encode character '\u21e2' in position 21: character maps to <undefined>

It seems that the character `⇢` is not allowed in yaml files, so I'll replace it locally with `->` instead. After doing it, let's check if it worked:

In [2]:
model = cobra.io.read_sbml_model("../model/g-thermo.xml")
cobra.io.save_yaml_model(model,"../model/g-thermo.yml")

Now the file can properly get stored, hooray! But there are still characters not compliant with `.yml`, as they are saved as `�`. After inspecting them, they all correspond to `’` in the original `.xml`, as in:

In [3]:
model.metabolites.pydx5p_c.notes

{'NAME': 'Pyridoxal 5’-phosphate; Pyridoxal phosphate; PLP; Pyridoxal 5-phosphate;'}

Sadly we cannot replace those characters with e.g. `'`, as they would get stored as `&apos;` in both the `.xml` and `.yml`. So we will just keep them as `’` in `.xml` and `�` in `.yml`, as they are the only characters that get stored like that.

## 2. Updating yaml from cobrapy

The final thing is to update our cobrapy fork to by default store the `.yml` file everytime the `.xml` is updated using `write_sbml_model`. This is done in commit [`b9b867f`](https://github.com/BenjaSanchez/cobrapy/commit/b9b867f3861362c3a3214c753a671992204dbfe9), and although not ideal (as the saving of each file should remain separate), as we are relying on this fork already it is the easiest way as it requires no changes on the user side (just to upgrade their cobra version). Eventually, after all the changes to make cobrapy & cobratoolbox fully compatible are integrated, we will switch to use the official cobrapy version, and then develop a wrapper function of the type `write_thermo_model` that saves both the `.xml` and `.yml` files. An issue will be open for this to remember to do it.

Let's test if the new addition works, by deleting the `.yml` file locally and then doing a normal I/O cycle:

In [4]:
model = cobra.io.read_sbml_model("../model/g-thermo.xml")
cobra.io.write_sbml_model(model,"../model/g-thermo.yml")

We can see that the `.yml` file gets re-created. Success!